<a href="https://colab.research.google.com/github/Classymotion/Smilegate/blob/main/gpt2_fine_tuning_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.7 MB/s eta 0:00:00


In [2]:
'''
qa_pairs = [
['Q: How to delete the account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to deactivate my account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How i can remove my nmbr A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to delete an account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.']
]
'''
qa_pairs = [
['Q: 빛보다 빨리 달리려면? A: 에너지가 많이 필요해'],
['Q: 미래로 가려면? A: 빛의 속도로 달리면 돼'],
['Q: 시간을 거꾸로 돌리려면 A: 그건 불가능해, 빛보다 빨리 달리면 몰라도'],
['Q: 시간은 소중한 거니 A: 당근당근 당연한 이야기야']
]



In [3]:
questions = '''
모델링 단계에서 어떤 요소들을 고려해야 하는지?
CAE 모델의 정확성을 확인하기 위해 어떤 검증 방법을 사용해야 하는지?
모델링에서 일어날 수 있는 오류와 해결책은 무엇인지?
모델을 개선하기 위해 어떤 수정 및 보완 작업을 해야 하는지?
분석에서 다양한 하중 조건을 고려하기 위한 방법은 무엇인지?
모델의 복잡성을 줄이기 위해 어떤 단순화 기법을 사용할 수 있는지?
분석 결과를 검증하기 위해 어떤 실험 및 시험 방법을 사용할 수 있는지?
분석에서 사용되는 소프트웨어 및 하드웨어의 성능을 최적화하기 위해 어떤 방법을 사용할 수 있는지?
모델링과 분석에서 생기는 비용과 시간을 줄이기 위한 방법은 무엇인지?
모델링과 분석에서 발생할 수 있는 잠재적 위험에 대해 어떻게 대처할 수 있는지?
OLED 모듈 제조 과정에서 가장 중요한 단계는 무엇인가요?
모듈 제조에 사용되는 기술적인 장비는 어떤 것들이 있나요?
OLED 모듈 제조에서 생기는 주요 문제점과 해결책은 무엇인가요?
사용되는 소재는 어떤 것들이 있나요?
제조에서 생기는 환경 문제와 이를 해결하기 위한 방법은 무엇인가요?
모듈 제조에서 생기는 생산 비용과 이를 줄이기 위한 방법은 무엇인가요?
인력 문제와 이를 해결하기 위한 방법은 무엇인가요?
제조에서 사용되는 기술적인 장비의 성능을 개선하기 위한 방법은 무엇인가요?
OLED 모듈에서 생기는 안전 문제와 이를 해결하기 위한 방법은 무엇인가요?
OLED 제조에서 발생하는 폐기물 문제와 이를 해결하기 위한 방법은 무엇인가요?
현재 불량이 발생하는 공정과 원인은 무엇인가요?
불량이 발생하는 제품의 종류와 빈도는 어떻게 되나요?
불량검출하는 방법과 검출율은 어떻게 되나요?
원인을 파악하기 위한 분석 도구와 방법은 무엇인가요?
작업자의 역할과 책임은 무엇인가요?
불량을 줄이기 위한 품질 관리 체계는 어떻게 구성되어 있나요?
교육 및 훈련 프로그램은 어떻게 운영되나요?
원인을 파악한 후 개선을 위한 대책은 무엇인가요?
불량을 예방하기 위한 정기적인 유지보수 및 점검 계획은 어떻게 되나요?
신속하게 대응하기 위한 비상 대책은 어떻게 마련되어 있나요?
조치를 취하기 위한 절차와 책임 분담 방법은 어떻게 되나요?
생산 중단 및 손실을 최소화하기 위한 방법은 무엇인가요?
원인 파악 및 대응을 위한 데이터 분석 방법과 도구는 무엇인가요?
공정 개선을 위한 R&D 활동은 어떻게 이루어지나요?
관련 부서 및 담당자들 간의 의사소통 방법은 어떻게 되나요?
작업자들의 자발적인 개선 제안을 촉진하는 방법은 무엇인가요?
KPI(성과지표)는 어떻게 설정되고 측정되나요?
원인에 따른 개선 대책을 수립하고 이를 실시하기 위한 프로젝트 관리 방법은 무엇인가요?
고객과의 커뮤니케이션 방법과 대응 방안은 어떻게 되나요?
제조 수율을 높이기 위해서 해야할 일은?
'''
answers = '''
모델링 단계에서는 입력 데이터, 모델 선택, 하이퍼파라미터 조정, 모델 평가 지표 등을 고려해야 한다.
CAE 모델의 정확성을 확인하기 위해는 실제 데이터와 비교해볼 수 있는 검증 데이터를 사용하거나, 교차 검증 방법을 사용할 수 있다.
모델링에서 일어날 수 있는 오류에는 과적합, 언더피팅, 데이터 편향 등이 있으며, 이를 해결하기 위해 데이터 수집과 전처리, 모델 복잡도 조절, 정규화 등을 고려해야 한다.
모델을 개선하기 위해는 새로운 데이터 수집 및 전처리, 하이퍼파라미터 튜닝, 더 좋은 모델 선택 등이 필요하다.
분석에서 다양한 하중 조건을 고려하기 위한 방법으로는 각 하중 조건에 대한 시뮬레이션을 진행하거나, 다양한 하중 조건에서 실제 실험을 진행하는 것이 있다.
모델의 복잡성을 줄이기 위한 단순화 기법으로는 feature selection, PCA 등이 있다.
분석 결과를 검증하기 위한 실험 및 시험 방법으로는 재현성 있는 실험, 교차 검증 등이 있다.
소프트웨어 및 하드웨어 성능을 최적화하기 위한 방법으로는 병렬 처리, GPU 가속화, 알고리즘 개선 등이 있다.
모델링 및 분석에서 생기는 비용과 시간을 줄이기 위한 방법으로는 적은 양의 데이터를 사용하는 등의 방법이 있다.
잠재적 위험 대처 방법으로는 데이터 보안 및 개인정보 보호, 모델 검증 및 검증 데이터 사용 등이 있다.
OLED 모듈 제조에서 가장 중요한 단계는 배기공정이다.
OLED 모듈 제조에 사용되는 기술적인 장비로는 증착장비, 마스크 정렬 장비, 노광 장비 등이 있다.
OLED 모듈 제조에서 주요 문제점으로는 불량 발생, 유기물 안정성, 수명 등이 있으며, 이를 해결하기 위해서는 불량 원인 분석, 안정성 개선, 수명 연장 기술 등이 필요하다.
사용되는 소재는 다양하지만 OLED 모듈 제조에서는 유기발광체, 전극, 캐패시터, 유리 등이 사용됩니다.
제조에서 생기는 환경 문제를 해결하기 위해 재활용, 에너지 효율적인 공정 등을 적용합니다.
모듈 제조에서 생기는 생산 비용을 줄이기 위해 자동화, 최적화된 생산 공정, 저렴한 재료 등을 사용합니다.
인력 문제를 해결하기 위해 교육 및 훈련 프로그램, 역량 강화 등을 진행합니다.
제조에서 사용되는 기술적인 장비의 성능을 개선하기 위해 IoT 기술, 빅데이터 분석, 인공지능 등을 적용합니다.
OLED 모듈에서 생기는 안전 문제를 해결하기 위해 안전 규정을 준수하고 안전한 설계를 적용합니다.
OLED 제조에서 발생하는 폐기물 문제를 해결하기 위해 재활용, 제로 웨이스트 생산 등의 방법을 적용합니다.
현재 불량이 발생하는 공정과 원인은 제품마다 다를 수 있으므로, 각 제품의 불량 발생 공정을 분석해야 합니다.
불량이 발생하는 제품의 종류와 빈도는 생산하는 제품의 종류와 생산량에 따라 다르므로 분석이 필요합니다.
불량검출하는 방법과 검출율은 시료 추출, 물리적 검사, 화학적 검사, 영상 분석 등의 방법을 사용하며, 검출율은 제품마다 다릅니다.
원인을 파악하기 위한 분석 도구와 방법은 통계적 분석, 품질 이력 추적, 페어링 분석 등이 있습니다.
작업자는 품질 관리에 책임을 지며, 역할은 제품 생산 과정에서 생기는 문제를 파악하고 조치하는 것입니다.
불량을 줄이기 위한 품질 관리 체계는 정해진 표준에 따라 설계되며, 관리 책임자와 절차, 문서화 등으로 이루어집니다.
교육 및 훈련 프로그램은 직무별로 계획되고 실시되며, 실적에 따라 평가 및 개선됩니다.
원인 파악을 위해 5W1H, Fishbone 등의 도구를 활용하며, 대책은 PDCA 사이클을 통해 수립됩니다.
정기적인 유지보수 및 점검 계획은 설비별로 수립되며, 이에 따라 전략적인 부품 관리가 이루어집니다.
비상 대책은 각 상황별로 미리 수립되어 있으며, 이에 따른 역할과 책임이 명확히 설정됩니다.
조치를 취하기 위한 절차와 책임 분담 방법은 SOP를 통해 정해지며, 이를 엄격하게 준수합니다.
생산 중단 및 손실을 최소화하기 위한 방법으로, 재고 관리, 공정 개선 등의 전략적인 대응이 이루어집니다.
데이터 분석 방법과 도구로는 통계학, Six Sigma, Minitab 등이 활용됩니다.
공정 개선을 위한 R&D 활동은 혁신적인 기술 개발과 실험 등의 다양한 방법을 활용합니다.
관련 부서 및 담당자들 간의 의사소통 방법은 회의, 보고서 등의 다양한 형태로 이루어집니다.
작업자들의 자발적인 개선 제안을 촉진하는 방법으로는 KIZ(칭찬, 격려, 제안) 제도 등이 활용됩니다.
KPI는 전략적 목표를 달성하기 위한 중요한 성과 지표로 설정되며, 정기적으로 측정 및 분석됩니다.
원인에 따른 개선 대책을 수립하고 이를 실시하기 위한 프로젝트 관리 방법으로는 PMBOK, Agile 등의 방법이 활용됩니다.
고객과의 커뮤니케이션 방법은 설명회, 고객 만족도 조사 등을 통해 이루어지며, 피드백을 수렴하여 적극적으로 대응합니다.
제조 수율을 높이기 위해서는 공정의 안정화, 공정 개선 등이 필요하며, 이를 위해 다양한 방법이 활용됩니다.
'''

In [4]:

'''
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
'''

from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")


'''
from transformers import AutoTokenizer, AutoModelForCausalLM 
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto'#, low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
'''

text = ''
'''
for question in qa_pairs:
  text += f'{question} {tokenizer.eos_token}'
'''

q_list = questions.lstrip().rstrip().splitlines()
a_list = answers.lstrip().rstrip().splitlines()
for i in range(40):
  text += f'Q:{q_list[i]} A:{a_list[i]} {tokenizer.eos_token}'

print(text)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Q:모델링 단계에서 어떤 요소들을 고려해야 하는지? A:모델링 단계에서는 입력 데이터, 모델 선택, 하이퍼파라미터 조정, 모델 평가 지표 등을 고려해야 한다. </s>Q:CAE 모델의 정확성을 확인하기 위해 어떤 검증 방법을 사용해야 하는지? A:CAE 모델의 정확성을 확인하기 위해는 실제 데이터와 비교해볼 수 있는 검증 데이터를 사용하거나, 교차 검증 방법을 사용할 수 있다. </s>Q:모델링에서 일어날 수 있는 오류와 해결책은 무엇인지? A:모델링에서 일어날 수 있는 오류에는 과적합, 언더피팅, 데이터 편향 등이 있으며, 이를 해결하기 위해 데이터 수집과 전처리, 모델 복잡도 조절, 정규화 등을 고려해야 한다. </s>Q:모델을 개선하기 위해 어떤 수정 및 보완 작업을 해야 하는지? A:모델을 개선하기 위해는 새로운 데이터 수집 및 전처리, 하이퍼파라미터 튜닝, 더 좋은 모델 선택 등이 필요하다. </s>Q:분석에서 다양한 하중 조건을 고려하기 위한 방법은 무엇인지? A:분석에서 다양한 하중 조건을 고려하기 위한 방법으로는 각 하중 조건에 대한 시뮬레이션을 진행하거나, 다양한 하중 조건에서 실제 실험을 진행하는 것이 있다. </s>Q:모델의 복잡성을 줄이기 위해 어떤 단순화 기법을 사용할 수 있는지? A:모델의 복잡성을 줄이기 위한 단순화 기법으로는 feature selection, PCA 등이 있다. </s>Q:분석 결과를 검증하기 위해 어떤 실험 및 시험 방법을 사용할 수 있는지? A:분석 결과를 검증하기 위한 실험 및 시험 방법으로는 재현성 있는 실험, 교차 검증 등이 있다. </s>Q:분석에서 사용되는 소프트웨어 및 하드웨어의 성능을 최적화하기 위해 어떤 방법을 사용할 수 있는지? A:소프트웨어 및 하드웨어 성능을 최적화하기 위한 방법으로는 병렬 처리, GPU 가속화, 알고리즘 개선 등이 있다. </s>Q:모델링과 분석에서 생기는 비용과 시간을 줄이기 위한 방법은 무엇인지? A:모델링 및 분석에서 생기는 비용과 시간을 줄이기 위한 방법으로는 적은 양

In [5]:
input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt')

In [6]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [8]:

model.train()

for epoch in range(1000):
  optimizer.zero_grad()
  loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
  loss.backward()
  optimizer.step()
  scheduler.step()

  if epoch % 100 == 0:
    print(f'Epoch {epoch}, Loss {loss.item()}')


RuntimeError: ignored

In [ ]:
model.eval()

In [ ]:
def generate_response(question):
  # Encode the question using the tokenizer
  #input_ids = tokenizer.encode(question + " <|question|>", add_special_tokens=False, return_tensors='pt').to(device)
  input_ids = tokenizer.encode(question + ":", add_special_tokens=False, return_tensors='pt').to(device)

  # Generate the answer using the model
  sample_output = model.generate(input_ids, do_sample=True, max_length=100, top_k=20, top_p=1.0)

  # Decode the generated answer using the tokenizer
  answer = tokenizer.decode(sample_output[0], skip_special_tokens=True)

  # Split the generated answer into individual sentences
  sentences = answer.split('. ')

  # Look for the sentence that contains the answer to the question
  for sentence in sentences:
      if question in sentence:
          return sentence

  # If no sentence contains the answer, return the full generated answer
  return answer

In [ ]:
for i in range(100):
  question = '문재인대통령은 오늘'
  response = generate_response(question)
  print(f'{response}')
